<a href="https://colab.research.google.com/github/Siddharth02003/CS6910_assignments/blob/main/CS6910_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt
import copy 
import random
import tensorflow as tf
from tqdm import tqdm


In [2]:
!pip install wandb -qqq
import wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 5.8 MB/s eta 0:00:00


In [3]:
def load_dataset():
    (X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
    return {
            'X_train': X_train,
            'Y_train': Y_train,
            'X_test': X_test,
            'Y_test': Y_test
        }

In [4]:
class_labels={0:'T-shirt',1:'Trouser',2:'Pullover',3:'Dress',4:'Coat',5:'Sandal',6:'Shirt',7:'Sneaker',8:'Bag',9:'Ankle_Boot'}

def label_to_name(label):
  if enumerate(label):
    l_names=[]
    for l in label:
      l_names.append(class_labels[l])
    return l_names
  else:
    return class_labels[label]

In [5]:
!wandb login --relogin
entity_name="siddharth-s"

project_name="FODL_Assignment_1"

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
def log_images():
  images=[]
  labels=[]
  dataset=load_dataset()
  X_train=dataset['X_train']
  y_train=dataset['Y_train']
  wandb.init(entity=entity_name,project=project_name, name="log_images")
  for i in range(100):
    if len(labels)==10:
      break
    if class_labels[y_train[i]] not in labels:
      images.append(X_train[i])
      labels.append(class_labels[y_train[i]])
  wandb.log({"Images": [wandb.Image(img, caption=caption) for img, caption in zip(images,labels)]})

In [7]:
class activation(): 
  def __init__(self,a):
    self.a=a

  def sigmoid(self,a):
    try:
      return (1.0/(1.0+np.exp(-a)))
    except:
      print("error")

  def relu(self,a):
    return (np.maximum(0,a))

  def tanh(self,a):
    return np.tanh(a)

  def softmax(self,a):
    try:
      return(np.exp(a)/np.sum(np.exp(a)))
    except:
      print("error")

  def sigmoid_derivative(self,x):
    return self.sigmoid(x)*(1-self.sigmoid(x))

  def tanh_derivative(self,x):
    return 1.0 -self.tanh(x)**2

  def relu_derivative(self,x):
    return 1. * (x>0)
     
  def softmax_derivative(self,x):
    return self.softmax(x) * (1-self.softmax(x))

  def derivative(self,x,activation):
    if activation == "sigmoid":
      return self.sigmoid_derivative(x)
    elif activation == "tanh":
      return self.tanh_derivative(x)
    elif activation == "relu":
      return self.relu_derivative(x)

In [8]:
class weights():
  def __init__(self,layers):
    self.layers=layers

  def Xavier(self,layers):
    params = {}
    for i in range(1,len(layers)):
       norm_xav=np.sqrt(6)/np.sqrt(layers[i]+layers[i-1])
       params["w"+str(i)]=np.random.randn(layers[i],layers[i-1])*norm_xav
       params["b"+str(i)]=np.zeros((layers[i],1))
    return params

  def Random(self,layers):
    params = {}
    for i in range(1,len(layers)):
       params["w"+str(i)]=0.01*np.random.randn(layers[i],layers[i-1])
       params["b"+str(i)]=0.01*np.random.randn(layers[i],1)
    return params

  def weight_init(self,init_type = "random"):
    params={}
    if(init_type=="xavier"):
      params = self.Xavier(self.layers)
    elif(init_type=="random"):
      params = self.Random(self.layers)
    else:
      print("invalid activation function")
    return params

In [9]:
def squared_loss(y, y_hat):
  error = np.sum(((y - y_hat)**2) / (2 * len(y)))
  return error

def CrossEntropy(y, y_hat):
  error = - np.sum( np.multiply(y , np.log(y_hat)))/len(y)
  return error

def loss_calc(loss_name, y, y_hat, lambd, layers, parameters):
  loss=0
  if(loss_name == "squared_loss"):
    loss=squared_loss(y, y_hat)
  elif(loss_name == "cross_entropy"):
    loss= CrossEntropy(y, y_hat)

  reg_loss = 0.0
  for i in range(len(layers)-1, 0, -1):
    reg_loss = reg_loss + (np.sum(parameters["w"+str(i)]))**2
  reg_loss = loss + ((lambd/(2*len(y)))*(reg_loss))
  return reg_loss

In [10]:
class network():
  def __init__(self,X,y,params,active,layers,loss_type):
    self.X=X
    self.y=y
    self.params=params
    self.active=active
    self.layers=layers
    self.loss_type=loss_type

  def forward_prop(self):
   out=copy.deepcopy(self.X)
   out=out.reshape(-1,1)
   h=[out]
   a=[out] 

   act=activation(a)

   if(self.active=="sigmoid"):
     for i in range(1,len(self.layers)-1):
       weights = self.params["w"+str(i)]
       biases = self.params["b"+str(i)]

       out = np.dot(weights,h[i-1])+biases
       a.append(out)
       post_a = act.sigmoid(out)
       h.append(post_a)
  
   elif(self.active=="tanh"):
     for i in range(1,len(self.layers)-1):
       weights=self.params["w"+str(i)]
       biases=self.params["b"+str(i)]
      
       out=np.dot(weights,h[i-1])+biases
       a.append(out)
       post_a=act.tanh(out)
       h.append(post_a)
  
   elif(self.active=="relu"):
     for i in range(1,len(self.layers)-1):
       weights=self.params["w"+str(i)]
       biases=self.params["b"+str(i)]
      
       out=np.dot(weights,h[i-1])+biases
       a.append(out)
       post_a=act.relu(out)
       h.append(post_a)       

   else:
     print("Invalid activation function") 
   weights=self.params["w"+str(len(self.layers)-1)]
   biases=self.params["b"+str(len(self.layers)-1)]
  
   out=np.dot(weights,h[len(self.layers)-2])+biases
   a.append(out)
   y_hat=act.softmax(out)
   h.append(y_hat)
   return h,a,y_hat

  def backward_prop(self,y,y_hat,h,a,params,layers):
    grad = {}
    act=activation(self.active)
    if self.loss_type == "squared_loss":
      grad["dh"+str(len(layers)-1)] = (y_hat - y)
      grad["da"+str(len(layers)-1)] = (y_hat - y) * act.softmax_derivative(a[len(layers)-1])

    elif self.loss_type == 'cross_entropy':
      grad["da"+str(len(layers)-1)] = -(y-y_hat)
      grad["dh"+str(len(layers)-1)] = -(y/y_hat)

    for i in range(len(layers)-1, 0, -1 ):
      grad["dw"+str(i)] = np.dot(grad["da"+str(i)], np.transpose(h[i-1]))
      grad["db"+str(i)] = grad["da"+str(i)]
      if i > 1:
        grad["dh"+str(i-1)] = np.dot(np.transpose(params["w"+str(i)]), grad["da"+str(i)])
        grad["da"+str(i-1)] = np.multiply(grad["dh" + str(i-1)], act.derivative(a[i-1],self.active))
    return grad

In [11]:
def accuracy_calc(res,y_t):
    acc=0.0   
    for x in range(len(res)):
      if(res[x].argmax()==y_t[x].argmax()):
        acc+=1
    acc=acc/len(y_t)
    return(acc*100)

In [12]:
def run_inference(X,y,parameters,activation,layers):
    result = []
    for i in range(len(X)):
      nn=network(X[i], y[i], parameters, activation, layers,"squared_loss")
      h,a,y_hat = nn.forward_prop()
      y_hat = y_hat.flatten()
      result.append(y_hat)
    return result

In [13]:
def calculate_grad(X, Y, parameters, activation, layers, loss_function):
  grads={}
  grads.clear() 
  for j in range(len(X)):
    y = np.reshape(Y[j], (-1,1))

    nn=network(X[j], y, parameters, activation, layers, loss_function)
    h,a,y_hat = nn.forward_prop()
    new_grads = nn.backward_prop(y,y_hat,h,a,parameters,layers)

    if j == 0:
      grads = copy.deepcopy(new_grads)
    else:
      for k in range(len(layers)-1,0,-1):
        grads["dw"+str(k)] += new_grads["dw"+str(k)]
        grads["db"+str(k)] += new_grads["db"+str(k)]
  return grads

In [14]:
def gradient_descent(X_train, y_train, eta, max_epochs, layers, mini_batch_size, lambd,loss_function, activation, parameters,optimiser,wandb_log=False):
  grads={}
  train_loss = []
  val_loss = []
  train_acc = []
  val_acc = []

  for t in tqdm(range(max_epochs)):
    for i in range(0, len(X_train), mini_batch_size):

      grads.clear()

      if str(optimiser) == "nesterovacc_gd":
        opt=optimiser(grads, eta, max_epochs,layers,mini_batch_size,lambd,parameters,i,t)
        param_lookahead,update_history=opt.paramlookahead()

      X = X_train[i:i + mini_batch_size]
      Y = y_train[i:i + mini_batch_size]
      
      if str(optimiser) == "nesterovacc_gd":
        grads = calculate_grad(X,Y,param_lookahead,activation,layers,loss_function)
      else: 
        grads = calculate_grad(X,Y,parameters,activation,layers,loss_function)

      opt=optimiser(grads, eta, max_epochs,layers,mini_batch_size,lambd,parameters,i,t)
      parameters=opt.get_params()
    
    #Calculating train loss 
    res = run_inference(X_train,y_train,parameters, activation, layers)
    train_err = loss_calc(loss_function,y_train,res,lambd,layers,parameters) 
    train_ac=accuracy_calc(res, y_train)
    train_loss.append(train_err)
    train_acc.append(train_ac)

    #Calculating validation loss
    res = run_inference(X_val, y_val, parameters, activation, layers)
    val_err = loss_calc(loss_function, y_val, res, lambd, layers, parameters )
    val_ac=accuracy_calc(res, y_val)
    val_loss.append(val_err)
    val_acc.append(val_ac)

    if(wandb_log==True):
      log_dict = {"Train_Accuracy": train_ac, "Validation_Accuracy": val_ac, \
                  "Train_Loss": train_err, "Validation_loss": val_err, "epoch": t}
                  
      wandb.log(log_dict)

  return parameters, train_acc, val_acc

In [15]:
class stochastic_gd():
  def __init__(self,grads, eta, max_epochs,layers,mini_batch_size,lambd,parameters,i,t):
    self.grads=grads
    self.eta=eta
    self.layers=layers
    self.mini_batch_size=mini_batch_size
    self.parameters=parameters
    self.lambd=lambd
    self.i=i
    self.t=t

  def get_params(self):
    for j in range(len(self.layers)-1,0,-1):
        self.parameters["w"+str(j)] = (1-((self.eta*self.lambd)/self.mini_batch_size))*self.parameters["w"+str(j)] - (self.eta * self.grads["dw"+str(j)])
        self.parameters["b"+str(j)] = self.parameters["b"+str(j)] - (self.eta * self.grads["db"+str(j)])
    return self.parameters

In [16]:
class momentum_gd():
  def __init__(self,grads, eta, max_epochs,layers,mini_batch_size,lambd,parameters,i,t):
    self.grads=grads
    self.eta=eta
    self.layers=layers
    self.mini_batch_size=mini_batch_size
    self.parameters=parameters
    self.lambd=lambd
    self.gamma=0.9
    self.i=i
    self.t=t

  def get_update_history(self):
    update_history={}
    for j in range(len(self.layers)-1, 0, -1):
          update_history["w"+str(j)] = self.eta*self.grads["dw"+str(j)]
          update_history["b"+str(j)] = self.eta*self.grads["db"+str(j)]
    for j in range(len(self.layers)-1, 0, -1):
          update_history["w"+str(j)] = (self.gamma*update_history["w"+str(j)]) + (self.eta*self.grads["dw"+str(j)])
          update_history["b"+str(j)] = (self.gamma*update_history["b"+str(j)]) + (self.eta*self.grads["db"+str(j)])
    return update_history

  def get_params(self):
    update_history=self.get_update_history()
    for j in range(len(self.layers)-1,0,-1):
        self.parameters["w"+str(j)] = (1-((self.eta*self.lambd)/self.mini_batch_size))*self.parameters["w"+str(j)] - update_history["w"+str(j)]
        self.parameters["b"+str(j)] = self.parameters["b"+str(j)] - update_history["b"+str(j)]
    return self.parameters

In [17]:
class nesterovacc_gd():
  def __init__(self,grads, eta, max_epochs,layers,mini_batch_size,lambd,parameters,i,t):
    self.grads=grads
    self.eta=eta
    self.layers=layers
    self.mini_batch_size=mini_batch_size
    self.parameters=parameters
    self.lambd=lambd
    self.i=i
    self.t=t

  def paramlookahead(self):
    update_history={}
    if self.i==0:
        param_lookahead = copy.deepcopy(self.parameters)
    else:
        for j in range(len(self.layers)-1, 0, -1):
          param_lookahead['w'+str(j)] = self.parameters['w'+str(j)] + (self.gamma*update_history["w"+str(j)])
    return param_lookahead,update_history

  def get_params(self,update_history):
    param_lookahead,update_history=self.paramlookahead()
    if self.i == 0 :
        for j in range(len(self.layers)-1, 0, -1):
          update_history["w"+str(j)] = self.eta*self.grads["dw"+str(j)]
          update_history["b"+str(j)] = self.eta*self.grads["db"+str(j)]
    else:
        for j in range(len(self.layers)-1, 0, -1):
          update_history["w"+str(j)] = (self.gamma*update_history["w"+str(j)]) + (self.eta*self.grads["dw"+str(j)])
          update_history["b"+str(j)] = (self.gamma*update_history["b"+str(j)]) + (self.eta*self.grads["db"+str(j)])
    for j in range(len(self.layers)-1,0,-1):
        self.parameters["w"+str(j)] = (1-((self.eta*self.lambd)/self.mini_batch_size))*self.parameters["w"+str(j)] - update_history["w"+str(j)]
        self.parameters["b"+str(j)] = self.parameters["b"+str(j)] - update_history["b"+str(j)]
    return self.parameters


In [18]:
class rmsprop():
  def __init__(self,grads, eta, max_epochs,layers,mini_batch_size,lambd,parameters,i,t):
    self.grads=grads
    self.eta=eta
    self.layers=layers
    self.mini_batch_size=mini_batch_size
    self.parameters=parameters
    self.lambd=lambd
    self.i=i
    self.beta = 0.9 
    self.epsilon=1e-8
    self.t=t

  def momenta(self):
    update_history={}
    v={}
    for i in range(len(self.layers)-1,0,-1):
      update_history["w"+str(i)]=np.zeros((self.layers[i],self.layers[i-1]))
      update_history["b"+str(i)]=np.zeros((self.layers[i],1))
    for i in range(len(self.layers)-1,0,-1):
      v["w"+str(i)]=np.zeros((self.layers[i],self.layers[i-1]))
      v["b"+str(i)]=np.zeros((self.layers[i],1))
    return v,update_history
     
  def get_params(self):
    v,update_history=self.momenta()
    for iq in range(len(self.layers)-1,0,-1):
        v["w"+str(iq)]=self.beta*v["w"+str(iq)]+(1-self.beta)*self.grads["dw"+str(iq)]**2
        v["b"+str(iq)]=self.beta*v["b"+str(iq)]+(1-self.beta)*self.grads["db"+str(iq)]**2     
        update_history["w"+str(iq)]=self.eta*np.multiply(np.reciprocal(np.sqrt(v["w"+str(iq)]+self.epsilon)),self.grads["dw"+str(iq)])
        update_history["b"+str(iq)]=self.eta*np.multiply(np.reciprocal(np.sqrt(v["b"+str(iq)]+self.epsilon)),self.grads["db"+str(iq)])
    for j in range(len(self.layers)-1,0,-1):
        self.parameters["w"+str(j)] = (1-((self.eta*self.lambd)/self.mini_batch_size))*self.parameters["w"+str(j)] - update_history["w"+str(j)]
        self.parameters["b"+str(j)] = self.parameters["b"+str(j)] - update_history["b"+str(j)]
    return self.parameters

In [19]:
class adam():
  def __init__(self,grads, eta, max_epochs,layers,mini_batch_size,lambd,parameters,i,t):
    self.grads=grads
    self.eta=eta
    self.layers=layers
    self.mini_batch_size=mini_batch_size
    self.parameters=parameters
    self.lambd=lambd
    self.i=i
    self.beta1=0.9 
    self.epsilon=1e-8
    self.beta2=0.999
    self.t=t
  
  def momenta(self):
    update_history={}
    v={}
    m={}
    for i in range(len(self.layers)-1,0,-1):
      update_history["w"+str(i)]=np.zeros((self.layers[i],self.layers[i-1]))
      update_history["b"+str(i)]=np.zeros((self.layers[i],1))
    for i in range(len(self.layers)-1,0,-1):
      v["w"+str(i)]=np.zeros((self.layers[i],self.layers[i-1]))
      v["b"+str(i)]=np.zeros((self.layers[i],1))
    for i in range(len(self.layers)-1,0,-1):
      m["w"+str(i)]=np.zeros((self.layers[i],self.layers[i-1]))
      m["b"+str(i)]=np.zeros((self.layers[i],1))
    return m,v,update_history
     
  def get_params(self):
    m,v,update_history=self.momenta()
    for iq in range(len(self.layers)-1,0,-1):
          m["w"+str(iq)]=self.beta1*m["w"+str(iq)]+(1-self.beta1)*self.grads["dw"+str(iq)]
          m["b"+str(iq)]=self.beta1*m["b"+str(iq)]+(1-self.beta1)*self.grads["db"+str(iq)]    
          v["w"+str(iq)]=self.beta2*v["w"+str(iq)]+(1-self.beta2)*(self.grads["dw"+str(iq)])**2
          v["b"+str(iq)]=self.beta2*v["b"+str(iq)]+(1-self.beta2)*(self.grads["db"+str(iq)])**2
          mw_hat=m["w"+str(iq)]/(1-np.power(self.beta1,self.t+1))
          mb_hat=m["b"+str(iq)]/(1-np.power(self.beta1,self.t+1))
          vw_hat=v["w"+str(iq)]/(1-np.power(self.beta2,self.t+1))
          vb_hat=v["b"+str(iq)]/(1-np.power(self.beta2,self.t+1))
          update_history["w"+str(iq)]=self.eta*np.multiply(np.reciprocal(np.sqrt(vw_hat+self.epsilon)),mw_hat)
          update_history["b"+str(iq)]=self.eta*np.multiply(np.reciprocal(np.sqrt(vb_hat+self.epsilon)),mb_hat)

    for j in range(len(self.layers)-1,0,-1):
          self.parameters["w"+str(j)] = (1-((self.eta*self.lambd)/self.mini_batch_size))*self.parameters["w"+str(j)] - update_history["w"+str(j)]
          self.parameters["b"+str(j)] = self.parameters["b"+str(j)] - update_history["b"+str(j)]
    return self.parameters

In [20]:
class nadam():
  def __init__(self,grads, eta, max_epochs,layers,mini_batch_size,lambd,parameters,i,t):
    self.grads=grads
    self.eta=eta
    self.layers=layers
    self.mini_batch_size=mini_batch_size
    self.parameters=parameters
    self.lambd=lambd
    self.i=i
    self.beta1=0.9 
    self.epsilon=1e-8
    self.beta2=0.999
    self.t=t
  
  def momenta(self):
    update_history={}
    v={}
    m={}
    for i in range(len(self.layers)-1,0,-1):
      update_history["w"+str(i)]=np.zeros((self.layers[i],self.layers[i-1]))
      update_history["b"+str(i)]=np.zeros((self.layers[i],1))
    for i in range(len(self.layers)-1,0,-1):
      v["w"+str(i)]=np.zeros((self.layers[i],self.layers[i-1]))
      v["b"+str(i)]=np.zeros((self.layers[i],1))
    for i in range(len(self.layers)-1,0,-1):
      m["w"+str(i)]=np.zeros((self.layers[i],self.layers[i-1]))
      m["b"+str(i)]=np.zeros((self.layers[i],1))
    return m,v,update_history
     
  def get_params(self):
    m,v,update_history=self.momenta()
    for iq in range(len(self.layers)-1,0,-1):
          m["w"+str(iq)]=self.beta1*m["w"+str(iq)]+(1-self.beta1)*self.grads["dw"+str(iq)]
          m["b"+str(iq)]=self.beta1*m["b"+str(iq)]+(1-self.beta1)*self.grads["db"+str(iq)]    
          v["w"+str(iq)]=self.beta2*v["w"+str(iq)]+(1-self.beta2)*(self.grads["dw"+str(iq)])**2
          v["b"+str(iq)]=self.beta2*v["b"+str(iq)]+(1-self.beta2)*(self.grads["db"+str(iq)])**2
          mw_hat=m["w"+str(iq)]/(1-np.power(self.beta1,self.t+1))
          mb_hat=m["b"+str(iq)]/(1-np.power(self.beta1,self.t+1))
          vw_hat=v["w"+str(iq)]/(1-np.power(self.beta2,self.t+1))
          vb_hat=v["b"+str(iq)]/(1-np.power(self.beta2,self.t+1))
          update_history["w"+str(iq)]=self.eta*np.multiply(np.reciprocal(np.sqrt(vw_hat+self.epsilon)),(self.beta1*mw_hat+(1-self.beta1)*self.grads["dw"+str(iq)]))*(1/(1-np.power(self.beta1,self.t+1)))
          update_history["b"+str(iq)]=self.eta*np.multiply(np.reciprocal(np.sqrt(vb_hat+self.epsilon)),(self.beta1*mb_hat+(1-self.beta1)*self.grads["db"+str(iq)]))*(1/(1-np.power(self.beta1,self.t+1)))

    for j in range(len(self.layers)-1,0,-1):
          self.parameters["w"+str(j)] = (1-((self.eta*self.lambd)/self.mini_batch_size))*self.parameters["w"+str(j)] - update_history["w"+str(j)]
          self.parameters["b"+str(j)] = self.parameters["b"+str(j)] - update_history["b"+str(j)]

    return self.parameters

Preparing Dataset

In [ ]:
from sklearn.model_selection import train_test_split
(train_x,train_y),(test_x,test_y)=fashion_mnist.load_data()
num_classes = 10
labels=['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']


#performing the train-validation split
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.1, random_state=40)
  

  
#storing the number of points in each set
  
X_train=np.zeros((len(train_x),784))
X_val=np.zeros((len(val_x),784))
X_test=np.zeros((len(test_x),784))
  
# converting the images into grayscale by normalizing
for i in range(len(train_x)):
  X_train[i]=(copy.deepcopy(train_x[i].flatten()))/255.0 
for i in range(len(val_x)):
  X_val[i]=(copy.deepcopy(val_x[i].flatten()))/255.0
for i in range(len(test_x)):
  X_test[i]=(copy.deepcopy(test_x[i].flatten()))/255.0
  
y_train = np.zeros((train_y.size, 10))
y_train[np.arange(train_y.size), train_y] = 1

y_val = np.zeros((val_y.size, 10))
y_val[np.arange(val_y.size), val_y] = 1

y_test = np.zeros((test_y.size, 10))
y_test[np.arange(test_y.size), test_y] = 1


4422102/4422102 [==============================] - 0s 0us/step


In [27]:
def train(X_train=X_train, y_train=y_train, layers=[784,16,10],wandb_log=True, learning_rate = 0.0001, initialization_type = "random", activation_function = "sigmoid", loss_function = "cross_entropy", mini_batch_Size = 32, max_epochs = 5, lambd = 0,optimization_function = momentum_gd):

  #Declaring the dictionary with the default hyperparameters
  config_defaults = {
      'number_hidden_layers': 2,
      'number_neurons': 32,
      'learning_rate': 0.001,
      'initialization_type': "xavier",
      'activation_function':'sigmoid',
      'mini_batch_size' : 64,
      'max_epochs': 5,
      'lambd': 0,
      'optimization_function': "nadam",
      'loss_function' : "cross_entropy"
  }

  # Initializing the wandb run
  wandb.init(config=config_defaults)
  config = wandb.config


  # Constructing the layers i.e., the architecture of our neural network
  layers = [784]
  for i in range(config.number_hidden_layers):
    layers = layers + [config.number_neurons]
  layers  = layers + [10]

  #Collecting all the hyperparameters from the wandb run
  learning_rate = config.learning_rate
  initialization_type = config.initialization_type
  activation_function = config.activation_function
  loss_function = config.loss_function
  mini_batch_size = config.mini_batch_size
  max_epochs = config.max_epochs
  lambd = config.lambd
  opt_fun = config.optimization_function
  hidden_layers=config.number_hidden_layers

  #Calling the respective hyperparameters
  if opt_fun == "adam":
    optimization_function = adam
  elif opt_fun == "nadam":
    optimization_function = nadam
  elif opt_fun == "stochastic_gd":
    optimization_function = stochastic_gd
  elif opt_fun == "momentum_gd":
    optimization_function = momentum_gd
  elif opt_fun == "nesterov-acc_gd":
    optimization_function = nesterovacc_gd
  elif opt_fun == "rmsprop":
    optimization_function = rmsprop
  else:
    print("Wrong optimization function")
    exit()


  name_run = str(hidden_layers) + "_" + initialization_type[0] + "_" + \
  activation_function[:4] + "_" + str(learning_rate) + "_" + opt_fun[:4]

  wandb.run.name = name_run
  wandb_log=True
  
  w=weights(layers)
  parameters = w.weight_init(init_type = initialization_type)
  parameters, train_acc, val_acc = gradient_descent(X_train, y_train,learning_rate, max_epochs, layers, mini_batch_Size, lambd, loss_function, activation_function, parameters,optimization_function,wandb_log)
  
  print("Training Accuracy:",train_acc[-1])
  print("Validation Accuracy:",val_acc[-1])
  
  wandb.run.save()
  wandb.run.finish()

In [ ]:
# layers=[784,32,10]
# wandb.init(entity_name,project_name)
# wandb_log=True
# train(X_train, y_train, layers,wandb_log)

In [ ]:
def do_sweep(entity_name,project_name):

  #Using bayes method for hyperparameter sweeps to curb the unnecessary configurations
  hyperparameters = {
      "learning_rate":{
        'values': [0.001, 0.0001]
      },

      "number_hidden_layers": {
          'values' : [3, 4, 5]
      },

      "number_neurons": {
        'values': [32, 64, 128]
      },

      "initialization_type": {
          'values' : ["xavier", "random"]
      },

      "activation_function": {
          'values': ["sigmoid", "tanh", "relu"]
      },

      "mini_batch_size": {
          'values': [16,32,64]
      },

      "max_epochs": {
          'values': [5, 10]
      },

      "lambd": {
          'values': [0, 0.0005, 0.5]
      },

      "optimization_function": {
          'values': ["stochastic_gd","momentum_gd","rmsprop","adam","nadam"]
      }

  }


  #Using bayes method for hyperparameter sweeps to curb the unnecessary configurations
  sweep_config = {
      'method' : 'bayes',
      'metric' :{
          'name': 'Validation_Accuracy',
          'goal': 'maximize'
      },
      'parameters': hyperparameters
  }

  sweep_id = wandb.sweep(sweep_config, entity=entity_name, project=project_name)
  wandb.agent(sweep_id, train)

In [ ]:
do_sweep(entity_name,project_name)

Create sweep with ID: pqng4xvs
Sweep URL: https://wandb.ai/siddharth-s/FODL_Assignment_1/sweeps/pqng4xvs


wandb: Agent Starting Run: 8ildkjcy with config:
wandb: 	activation_function: relu
wandb: 	initialization_type: xavier
wandb: 	lambd: 0.5
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 10
wandb: 	mini_batch_size: 16
wandb: 	number_hidden_layers: 3
wandb: 	number_neurons: 64
wandb: 	optimization_function: adam
wandb: Currently logged in as: na20b064 (siddharth-s). Use `wandb login --relogin` to force relogin


100%|██████████| 10/10 [05:52<00:00, 35.24s/it]
wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Training Accuracy: 84.85555555555555
Validation Accuracy: 84.43333333333334


Train_Accuracy,▁▄▆▇▇▇▇███
Train_Loss,▁▁▁▂▃▄▅▆▇█
Validation_Accuracy,▁▃▆▆▆▇▇▇██
Validation_loss,▁▁▂▃▄▄▅▆▇█
epoch,▁▂▃▃▄▅▆▆▇█
Train_Accuracy,84.85556
Train_Loss,1.06951
Validation_Accuracy,84.43333
Validation_loss,5.57614
epoch,9


...

100%|██████████| 10/10 [10:10<00:00, 61.05s/it]

Training Accuracy: 10.048148148148147
Validation Accuracy: 9.566666666666666


Train_Accuracy,█▁▁▁▁▁▁▁▁▁
Validation_Accuracy,█▁▁▁▁▁▁▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
Train_Accuracy,10.04815
Train_Loss,nan
Validation_Accuracy,9.56667
Validation_loss,nan
epoch,9


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: t5h73ukj with config:
wandb: 	activation_function: tanh
wandb: 	initialization_type: random
wandb: 	lambd: 0.0005
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 5
wandb: 	mini_batch_size: 64
wandb: 	number_hidden_layers: 4
wandb: 	number_neurons: 128
wandb: 	optimization_function: adam


  0%|          | 0/5 [00:00<?, ?it/s]

Plotting Confusion Matrix


In [ ]:
# Hyperparameters with highest validation accuracy: 86.833
activation_function= "tanh"
initialization_type= "xavier"
lambd=0
learning_rate=0.001
max_epochs=5
mini_batch_size=16
number_hidden_layers=5
number_neurons=128
optimization_function=nadam
loss_function = "cross_entropy"

layers=[784]
for i in range(number_hidden_layers):
  layers=layers+[number_neurons]
layers=layers+[10]


config_confmat = {
      'number_hidden_layers': 5,
      'number_neurons': 128,
      'learning_rate': 0.0001,
      'initialization_type': "xavier",
      'activation_function':'tanh',
      'mini_batch_size' : 16,
      'max_epochs': 5,
      'lambd': 0,
      'optimization_function': "nadam"
  }

wandb.init(config=config_confmat,entity=entity_name, project=project_name)
wandb.run.name="Confusion_matrix"
wandb_log=True

w=weights(layers)
parameters = w.weight_init(init_type = initialization_type)
parameters, train_acc, val_acc = gradient_descent(X_train, y_train,learning_rate, max_epochs, layers, mini_batch_size, lambd, loss_function, activation_function, parameters,optimization_function,wandb_log)
res = run_inference(X_test,y_test, parameters, activation_function, layers)


accuracy=accuracy_calc(res,y_test)
print("Test Accuracy:",accuracy)
#converting 1 hot labels to class ids

y=[]

for i in range(len(y_test)):
  y.append(y_test[i].argmax())

y_hat=[]

for i in range(len(res)):
  y_hat.append(res[i].argmax())

wandb.log({"conf_mat":wandb.plot.confusion_matrix(preds=y_hat,y_true=y,class_names=labels),"Test Accuracy": accuracy}) 
    
wandb.run.save()
wandb.run.finish()



100%|██████████| 5/5 [09:48<00:00, 117.74s/it]


Test Accuracy: 80.01


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Test Accuracy,▁
Train_Accuracy,▁▅▆▆█
Train_Loss,█▁▂▃▁
Validation_Accuracy,▁▅▆▆█
Validation_loss,█▁▂▃▁
epoch,▁▃▅▆█
Test Accuracy,80.01
Train_Accuracy,81.25926
Train_Loss,0.91054
Validation_Accuracy,81.13333
Validation_loss,0.93966


Cross Entropy vs Mean Squared Error Comparison

In [ ]:
def compare_loss(entity_name,project_name):


  #We fix all hyperparameters except for the loss function 
  hyperparameters = {
      "learning_rate":{
        'values': [0.0001]
      },

      "number_hidden_layers": {
          'values' : [5]
      },

      "number_neurons": {
        'values': [128]
      },

      "initialization_type": {
          'values' : ["xavier"]
      },

      "activation_function": {
          'values': ["tanh"]
      },

      "mini_batch_size": {
          'values': [16]
      },

      "max_epochs": {
          'values': [5]
      },

      "loss_function": {
          'values' : ["cross_entropy","squared_loss"]      
          
      },

      "lambd": {
          'values': [0.0005]
      },

      "optimization_function": {
          'values': ["adam"]
      }

  }


  sweep_config = {
      'method' : 'bayes',
      'metric' :{
          'name': 'Validation_Accuracy',
          'goal': 'maximize'
      },
      'parameters': hyperparameters
  }

  sweep_id = wandb.sweep(sweep_config, entity=entity_name, project=project_name)
  wandb.agent(sweep_id, train)

In [ ]:
compare_loss(entity_name,project_name)

Create sweep with ID: lzx4xd41
Sweep URL: https://wandb.ai/siddharth-s/FODL_Assignment_1/sweeps/lzx4xd41


wandb: Agent Starting Run: dze0k74q with config:
wandb: 	activation_function: tanh
wandb: 	initialization_type: xavier
wandb: 	lambd: 0.0005
wandb: 	learning_rate: 0.0001
wandb: 	loss_function: squared_loss
wandb: 	max_epochs: 5
wandb: 	mini_batch_size: 16
wandb: 	number_hidden_layers: 5
wandb: 	number_neurons: 128
wandb: 	optimization_function: adam


100%|██████████| 5/5 [03:55<00:00, 47.02s/it]

Training Accuracy: 88.20185185185186
Validation Accuracy: 86.11666666666666


Train_Accuracy,▁▅▆▇█
Train_Loss,█▄▃▂▁
Validation_Accuracy,▁▅▇▇█
Validation_loss,█▄▃▂▁
epoch,▁▃▅▆█
Train_Accuracy,88.20185
Train_Loss,0.08676
Validation_Accuracy,86.11667
Validation_loss,0.09917
epoch,4


...

Train_Accuracy,▁▄▆▇█
Train_Loss,█▅▃▂▁
Validation_Accuracy,▁▄▆▇█
Validation_loss,█▄▃▂▁
epoch,▁▃▅▆█
Train_Accuracy,87.49259
Train_Loss,0.3546
Validation_Accuracy,86.33333
Validation_loss,0.40283
epoch,4


wandb: Agent Starting Run: 0mfmethm with config:
wandb: 	activation_function: tanh
wandb: 	initialization_type: xavier
wandb: 	lambd: 0.0005
wandb: 	learning_rate: 0.0001
wandb: 	loss_function: squared_loss
wandb: 	max_epochs: 5
wandb: 	mini_batch_size: 16
wandb: 	number_hidden_layers: 5
wandb: 	number_neurons: 128
wandb: 	optimization_function: adam


 20%|██        | 1/5 [01:03<04:15, 63.91s/it]


Train_Accuracy,▁
Train_Loss,▁
Validation_Accuracy,▁
Validation_loss,▁
epoch,▁
Train_Accuracy,84.19444
Train_Loss,0.11283
Validation_Accuracy,83.56667
Validation_loss,0.11695
epoch,0


wandb: Sweep Agent: Waiting for job.
wandb: Ctrl + C detected. Stopping sweep.


MNIST Trials


In [26]:
# Preparing Dataset 
from sklearn.model_selection import train_test_split
from keras.datasets import mnist

(train_x,train_y),(test_x,test_y)=mnist.load_data()
num_classes = 10
labels=np.arange(0,10,1)


#performing the train-validation split
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.1, random_state=40)
  

  
#storing the number of points in each set
  
X_train=np.zeros((len(train_x),784))
X_val=np.zeros((len(val_x),784))
X_test=np.zeros((len(test_x),784))
  
# converting the images into grayscale by normalizing
for i in range(len(train_x)):
  X_train[i]=(copy.deepcopy(train_x[i].flatten()))/255.0 
for i in range(len(val_x)):
  X_val[i]=(copy.deepcopy(val_x[i].flatten()))/255.0
for i in range(len(test_x)):
  X_test[i]=(copy.deepcopy(test_x[i].flatten()))/255.0
  
y_train = np.zeros((train_y.size, 10))
y_train[np.arange(train_y.size), train_y] = 1

y_val = np.zeros((val_y.size, 10))
y_val[np.arange(val_y.size), val_y] = 1

y_test = np.zeros((test_y.size, 10))
y_test[np.arange(test_y.size), test_y] = 1


11490434/11490434 [==============================] - 0s 0us/step


In [28]:
#Configuration 1

activation_function= "tanh"
initialization_type= "xavier"
lambd=0
learning_rate=0.001
max_epochs=10
mini_batch_size=16
number_hidden_layers=5
number_neurons=128
optimization_function=nadam
loss_function = "cross_entropy"

layers=[784]
for i in range(number_hidden_layers):
  layers=layers+[number_neurons]
layers=layers+[10]


config_confmat = {
      'number_hidden_layers': 5,
      'number_neurons': 128,
      'learning_rate': 0.0001,
      'initialization_type': "xavier",
      'activation_function':'tanh',
      'mini_batch_size' : 16,
      'max_epochs': 5,
      'lambd': 0,
      'optimization_function': "nadam"
  }

wandb.init(config=config_confmat,entity=entity_name, project=project_name)
wandb.run.name="MNIST_trials"
wandb_log=True

w=weights(layers)
parameters = w.weight_init(init_type = initialization_type)
parameters, train_acc, val_acc = gradient_descent(X_train, y_train,learning_rate, max_epochs, layers, mini_batch_size, lambd, loss_function, activation_function, parameters,optimization_function,wandb_log)
res = run_inference(X_test,y_test, parameters, activation_function, layers)


accuracy=accuracy_calc(res,y_test)
print("Test Accuracy:",accuracy)

wandb: Currently logged in as: na20b064 (siddharth-s). Use `wandb login --relogin` to force relogin


100%|██████████| 10/10 [16:24<00:00, 98.50s/it]


Test Accuracy: 95.39999999999999


In [30]:
#Configuration 2

activation_function= "tanh"
initialization_type= "xavier"
lambd=0.0005
learning_rate=0.001
max_epochs=7
mini_batch_size=32
number_hidden_layers=5
number_neurons=128
optimization_function=rmsprop
loss_function = "cross_entropy"

layers=[784]
for i in range(number_hidden_layers):
  layers=layers+[number_neurons]
layers=layers+[10]


config_confmat = {
      'number_hidden_layers': 5,
      'number_neurons': 128,
      'learning_rate': 0.0001,
      'initialization_type': "xavier",
      'activation_function':'tanh',
      'mini_batch_size' : 32,
      'max_epochs': 7,
      'lambd': 0.0005,
      'optimization_function': "rmsprop"
  }

wandb.init(config=config_confmat,entity=entity_name, project=project_name)
wandb.run.name="MNIST_trials"
wandb_log=True

w=weights(layers)
parameters = w.weight_init(init_type = initialization_type)
parameters, train_acc, val_acc = gradient_descent(X_train, y_train,learning_rate, max_epochs, layers, mini_batch_size, lambd, loss_function, activation_function, parameters,optimization_function,wandb_log)
res = run_inference(X_test,y_test, parameters, activation_function, layers)


accuracy=accuracy_calc(res,y_test)
print("Test Accuracy:",accuracy)

Train_Accuracy,▁
Train_Loss,▁
Validation_Accuracy,▁
Validation_loss,▁
epoch,▁
Train_Accuracy,94.01481
Train_Loss,0.27383
Validation_Accuracy,93.86667
Validation_loss,0.27809
epoch,0


100%|██████████| 7/7 [09:21<00:00, 80.26s/it]


Test Accuracy: 95.15


In [31]:
#Configuration 3

activation_function= "tanh"
initialization_type= "xavier"
lambd=0
learning_rate=0.001
max_epochs=7
mini_batch_size=64
number_hidden_layers=5
number_neurons=64
optimization_function=adam
loss_function = "cross_entropy"

layers=[784]
for i in range(number_hidden_layers):
  layers=layers+[number_neurons]
layers=layers+[10]


config_confmat = {
      'number_hidden_layers': 5,
      'number_neurons': 64,
      'learning_rate': 0.0001,
      'initialization_type': "xavier",
      'activation_function':'tanh',
      'mini_batch_size' : 64,
      'max_epochs': 7,
      'lambd': 0,
      'optimization_function': "adam"
  }

wandb.init(config=config_confmat,entity=entity_name, project=project_name)
wandb.run.name="MNIST_trials"
wandb_log=True

w=weights(layers)
parameters = w.weight_init(init_type = initialization_type)
parameters, train_acc, val_acc = gradient_descent(X_train, y_train,learning_rate, max_epochs, layers, mini_batch_size, lambd, loss_function, activation_function, parameters,optimization_function,wandb_log)
res = run_inference(X_test,y_test, parameters, activation_function, layers)


accuracy=accuracy_calc(res,y_test)
print("Test Accuracy:",accuracy)

Train_Accuracy,▁▆▇███▇
Train_Loss,█▃▃▁▁▁▂
Validation_Accuracy,▁▆████▇
Validation_loss,▇▃▃▃▁▃█
epoch,▁▂▃▅▆▇█
Train_Accuracy,96.24444
Train_Loss,0.24194
Validation_Accuracy,95.41667
Validation_loss,0.33417
epoch,6


100%|██████████| 7/7 [05:21<00:00, 45.94s/it]


Test Accuracy: 96.34
